In [ ]:
!pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import opendatasets as op
import sys

In [ ]:
import tensorflow.keras as keras
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import LSTM, Embedding, Dense, Activation, Flatten, Reshape, Dropout,add,Input
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.utils import img_to_array,load_img


{"username":"g2savage","key":"6fd0119816dd058b2ad81402a9dff717"}

In [ ]:
op.download('https://www.kaggle.com/datasets/kunalgupta2616/flickr-8k-images-with-captions')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: g2savage
Your Kaggle Key: ··········


100%|██████████| 1.04G/1.04G [00:13<00:00, 85.3MB/s]


In [ ]:
captions=pd.read_csv('/content/flickr-8k-images-with-captions/captions.txt')

In [ ]:
captions['caption']=captions['caption'].map(lambda x:re.sub(r'[^\w\s]', '', x).lower())

In [ ]:
captions['caption']='<start>'+' '+captions['caption']+' '+'<end>'

In [ ]:
token=Tokenizer(num_words=1660)
token.fit_on_texts(captions['caption'])

In [ ]:
captions['caption']=token.texts_to_sequences(captions['caption'])

In [ ]:
v=np.array(captions)

In [ ]:
caps=[]
names=[]
target=[]
for cap in range(40455):
    for t in range(1,1+len(v[cap,1])): 
        caps.append(v[cap,1][:t])
        names.append(v[cap,0])
        try:
            target.append(v[cap,1][t])
        except:
            target.append(0)


In [ ]:
caps=pad_sequences(caps,maxlen=30,)#padding='post',truncating='post')

In [ ]:
caps[1].shape

(30,)

In [ ]:
casp=[np.expand_dims(o,axis=1) for o in caps]

casp[1].shape

(30, 1)

In [ ]:
np.array(casp).shape

(416795, 1, 1, 30)

In [ ]:
from keras import Sequential

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=([30,1])))
model.add(Dense(1660))
model.add(Activation('softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
pd.

(1, 30)

In [ ]:

model.fit(casp, np.array(target), validation_split=0.05, batch_size=128, epochs=2,)

ValueError: ignored

In [ ]:
casp[1]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 3, 1]], dtype=int32)

In [ ]:
modelc = VGG16()
modelc = Model(inputs=modelc.inputs, outputs=modelc.layers[-2].output)
def extract_features(filename):

    # load an image from file /content/flickr-8k-images-with-captions/Images
    filename =filename

    # load the image and convert it into target size of 224*224
    image = load_img(filename, target_size=(224, 224))

    # convert the image pixels to a numpy array
    image = img_to_array(image)

    # reshape data for the model
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))

    # prepare the image for the VGG model
    image = preprocess_input(image)

    # get features
    feature = modelc.predict(image, verbose=0)
    np.reshape(feature,feature.shape[1])
    return np.reshape(feature,feature.shape[1])

553467096/553467096 [==============================] - 8s 0us/step


In [ ]:
features = dict()
i=0
for name in captions['image'].unique():
    u='/content/flickr-8k-images-with-captions/Images'+'/'+f'{name}'
    features[name]=extract_features(u)
    i+=1
    if i == 10:
        break

In [ ]:
features=np.array(list(features.values()))

In [ ]:
features[1].shape

(4096,)

In [ ]:
tart=np.array(pd.get_dummies(target))

In [ ]:
tart[1].shape

(1660,)

In [ ]:
tart[1].shape

(1,)

In [ ]:
caps[1].shape

(30,)

In [ ]:
def data_generator(nbatch):
    X1, X2, y = list(), list(), list()
    n=0
    for n in range(nbatch):
        if not np.array_equal(casp[n+1], casp[0]):
            X1.append(np.expand_dims(features[names[2]],axis=0))
            X2.append(casp[n])
            y.append(target[n])

        else:
            continue
        if n==nbatch-1:
            return X1,X2,y
            break

In [ ]:
features['1000268201_693b08cb0e.jpg'].shape

(4096,)

In [ ]:
inputs1 = Input(shape=(4096))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)

# partial caption sequence model
inputs2 = Input(shape=(30))
se1 = Embedding(1660,65, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)
# decoder (feed forward) model
decoder1 = add([fe2,se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(1660, activation='softmax')(decoder2)

# merge the two input models
model = Model(inputs=[inputs1, inputs2], outputs=outputs)

In [ ]:
import tensorflow as tf


In [ ]:
model.compile(optimizer='adam',loss = 'categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit([pd.DataFrame(features),pd.DataFrame(caps[0:10])],pd.DataFrame(tar))

1/1 [==============================] - 9s 9s/step - loss: 7.3515 - accuracy: 0.0000e+00


In [ ]:
tar=tart[0:10]

In [ ]:
pd.DataFrame(features).shape

(10, 4096)

In [ ]:
model.predict([pd.DataFrame(features),pd.DataFrame(caps[0:10])])

1/1 [==============================] - 2s 2s/step


array([[0.00067761, 0.00106625, 0.00032135, ..., 0.00064147, 0.00158292,
        0.0004994 ],
       [0.0005701 , 0.00068808, 0.00043019, ..., 0.00065197, 0.00075954,
        0.0005196 ],
       [0.00063543, 0.00151077, 0.00016851, ..., 0.00028324, 0.00065811,
        0.00044751],
       ...,
       [0.00061045, 0.00105826, 0.00030668, ..., 0.00066793, 0.0005373 ,
        0.00068207],
       [0.001007  , 0.00055429, 0.00029197, ..., 0.00079589, 0.00138528,
        0.00043874],
       [0.00122307, 0.0007826 , 0.00021299, ..., 0.00054944, 0.00080067,
        0.00067586]], dtype=float32)

In [ ]:
x1=pd.DataFrame(caps[0:10])

In [ ]:
x2=pd.DataFrame(features).T

In [ ]:
x3

,0,1,2,3,4,5,6,7,8,9,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
model.fit([x2.to_array(),x1.to_array()],[1,2,3,4,4,5,6,7,7,8],epochs=2,verbose=1)

AttributeError: ignored

In [ ]:
x1.values()

TypeError: ignored

In [ ]:
from keras import Sequential

In [ ]:

input1=(Input(shape=(2)))
input2=(Input(shape=(3)))
input2=Dense(2, activation='relu')(input2)
model2=add([input1,input2])
model2=Dense(12, activation='relu')(model2)
model2=Dense(8, activation='relu')(model2)
model2=Dense(1, activation='sigmoid')(model2)
j=Model(inputs=[input1,input2],outputs=model2)

In [ ]:
j.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
k=np.array([[1,2],
            [1,2]])
l=np.array([[1,2,3],
            [1,2,3]])

In [ ]:
j.predict([k,l])

InvalidArgumentError: ignored

In [ ]:
j.fit([k,k],[1,2])

ValueError: ignored

In [ ]:
y=[1,1]

In [ ]:
k=np.array([[1,2,3],
            [1,2,3]])
l=k=np.array([[1,2],
            [1,2]])

In [ ]:
j.predict[l,k]

TypeError: ignored

In [ ]:
j.fit(X, y, epochs=150, batch_size=10)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: ignored